<a href="https://colab.research.google.com/github/poolaNaveen/PySpark/blob/main/pyspark7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Before running a any Delta format file we need to install all configuration.
#2.Delta tables are used because they bring ACID transactions, version control, schema enforcement, fast performance, and safe concurrent writing to data lakes—something normal Parquet/CSV files cannot do.
#3.!apt-get install default-jdk -y
#4.!pip install pyspark delta-spark
#5.from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("DeltaLocal") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [ ]:
!apt-get install default-jdk -y



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core default-jdk-headless default-jre default-jre-headless
  fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libxcomposite1 libxt-dev libxtst6 libxxf86dga1
  openjdk-11-jdk openjdk-11-jdk-headless openjdk-11-jre
  openjdk-11-jre-headless session-migration x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  at-spi2-core default-jdk default-jdk-headless default-jre
  default-jre-headless fonts-dejavu-core fonts-dejavu-extra
  gsettings-desktop-schemas libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jn

In [ ]:
!pip install pyspark delta-spark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 16.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813800 sha256=2227c6e55509a4c567bf16b5159179f822080ae8f4d9f74f9e2d64fc9d3c5448
  Stored in directory: /root/.cache/pip/wheels/31/9f/68/f89fb34ccd886909be7d0e390eaaf97f21efdf540c0ee8dbcd
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [ ]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

spark = SparkSession.builder \
    .appName("LocalDelta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.schema.autoMerge.enabled", 'true') \
    .getOrCreate()


data = [(1, "Charan"), (2, "Ram")]
df = spark.createDataFrame(data, ["id", "name"])

df.write.format("delta").mode("overwrite").save("emp2")

existing_table = DeltaTable.forPath(spark, "emp2")
new_data = spark.createDataFrame([(1, "Charan C"), (4, "Suresh")], ["id", "name"])

existing_table.alias("e").merge(
    new_data.alias("n"),
    "e.id = n.id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

df = spark.read.format("delta").load("emp2")
df.show()

+---+--------+
| id|    name|
+---+--------+
|  1|Charan C|
|  4|  Suresh|
|  2|     Ram|
+---+--------+

